In [1]:
from matplotlib import pyplot as plt
import numpy as np

from skimage.io import imread


from skimage.measure import label

from faim_ipa.utils import get_git_root
import sys
import torch
sys.path.append(str(get_git_root()))

In [2]:
from source.rdcnet.model import RDCNet2d
from source.data.BBBC039 import BBBC039

/home/tibuch/pytorch-rdc-net/.pixi/envs/jupyter/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
torch.set_float32_matmul_precision('high')
dm = BBBC039(
    img_dir=get_git_root() / "raw_data" / "BBBC039" / "images",
    label_dir=get_git_root() / "raw_data" / "BBBC039" / "masks",
)

Train: 160, Val: 20, Test: 20


In [4]:
model = RDCNet2d.load_from_checkpoint(get_git_root() / "runs/BBBC039-testing/lightning_logs/version_7/checkpoints/rdcnet-epoch=84-mean_true_score=0.85.ckpt")

In [5]:
test_dl = dm.test_dataloader()

In [6]:
from tqdm.notebook import tqdm

In [7]:
from source.matching import matching

In [8]:
model = model.to('cuda')

In [9]:
preds = []
gts = []
inputs = []
for x, y in tqdm(test_dl):
    inputs.append(x[0, 0])
    preds.append(model.predict_instances(x)[0,0])
    gts.append(y[0, 0])

  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
from napari import Viewer

In [11]:
viewer = Viewer()

In [14]:
i = 12
viewer.layers.clear()
viewer.add_image(inputs[i], contrast_limits=np.quantile(inputs[i], [0.03, 0.998]))
viewer.add_labels(gts[i], name='GT', opacity=0.2)
viewer.add_labels(preds[i], name='Prediction')
viewer.layers[-1].contour = 2

m = matching(gts[i].cpu().numpy(), preds[i])
print(m.mean_true_score)

0.8428487141927083
